In [1]:
!pip install kaggle

In [2]:
!pip install dlt[duckdb]

In [3]:
!pip install --upgrade pyarrow

  Using cached pyarrow-19.0.1-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
Using cached pyarrow-19.0.1-cp312-cp312-manylinux_2_28_x86_64.whl (42.1 MB)
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 17.0.0
    Uninstalling pyarrow-17.0.0:
      Successfully uninstalled pyarrow-17.0.0


In [4]:
!pip install dlt[bigquery]

  Using cached pyarrow-17.0.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
Using cached pyarrow-17.0.0-cp312-cp312-manylinux_2_28_x86_64.whl (39.9 MB)
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 19.0.1
    Uninstalling pyarrow-19.0.1:
      Successfully uninstalled pyarrow-19.0.1


In [5]:
import kaggle,pandas as pd,dlt

In [6]:
kaggle.api.authenticate()

In [7]:
kaggle.api.dataset_download_files("shubhambathwal/flight-price-prediction",path='./data',unzip=True)

Dataset URL: https://www.kaggle.com/datasets/shubhambathwal/flight-price-prediction


In [8]:
df = kaggle.api.dataset_list_files("shubhambathwal/flight-price-prediction").files
print(kaggle.api.dataset_list_files("shubhambathwal/flight-price-prediction").files)

[{"ref": "", "datasetRef": "", "ownerRef": "", "name": "Clean_Dataset.csv", "creationDate": "2022-02-25T17:58:21.640Z", "description": "", "fileType": "", "url": "", "totalBytes": 24683279, "columns": []}, {"ref": "", "datasetRef": "", "ownerRef": "", "name": "business.csv", "creationDate": "2022-02-25T17:58:21.161Z", "description": "", "fileType": "", "url": "", "totalBytes": 9868748, "columns": []}, {"ref": "", "datasetRef": "", "ownerRef": "", "name": "economy.csv", "creationDate": "2022-02-25T17:58:21.551Z", "description": "", "fileType": "", "url": "", "totalBytes": 21096149, "columns": []}]


In [9]:
df[:5]

[{"ref": "", "datasetRef": "", "ownerRef": "", "name": "Clean_Dataset.csv", "creationDate": "2022-02-25T17:58:21.640Z", "description": "", "fileType": "", "url": "", "totalBytes": 24683279, "columns": []},
 {"ref": "", "datasetRef": "", "ownerRef": "", "name": "business.csv", "creationDate": "2022-02-25T17:58:21.161Z", "description": "", "fileType": "", "url": "", "totalBytes": 9868748, "columns": []},
 {"ref": "", "datasetRef": "", "ownerRef": "", "name": "economy.csv", "creationDate": "2022-02-25T17:58:21.551Z", "description": "", "fileType": "", "url": "", "totalBytes": 21096149, "columns": []}]

In [10]:
CleanData = pd.read_csv('./data/Clean_Dataset.csv')
BusinessData = pd.read_csv('./data/business.csv')
EconomyData= pd.read_csv('./data/economy.csv')

In [11]:
CleanData.info()
print(CleanData)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300153 entries, 0 to 300152
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        300153 non-null  int64  
 1   airline           300153 non-null  object 
 2   flight            300153 non-null  object 
 3   source_city       300153 non-null  object 
 4   departure_time    300153 non-null  object 
 5   stops             300153 non-null  object 
 6   arrival_time      300153 non-null  object 
 7   destination_city  300153 non-null  object 
 8   class             300153 non-null  object 
 9   duration          300153 non-null  float64
 10  days_left         300153 non-null  int64  
 11  price             300153 non-null  int64  
dtypes: float64(1), int64(3), object(8)
memory usage: 27.5+ MB
        Unnamed: 0   airline   flight source_city departure_time stops  \
0                0  SpiceJet  SG-8709       Delhi        Evening  zero   
1     

In [12]:
BusinessData.head(10)
BusinessData= pd.DataFrame(BusinessData)
BusinessData['stop']= BusinessData['stop'].str.replace(r'\n.*','',regex=True)

In [13]:
BusinessData['date']=pd.to_datetime(BusinessData['date'], format="%d-%m-%Y", dayfirst=True)

In [14]:
print(BusinessData)

            date    airline ch_code  num_code dep_time     from time_taken  \
0     2022-02-11  Air India      AI       868    18:00    Delhi    02h 00m   
1     2022-02-11  Air India      AI       624    19:00    Delhi    02h 15m   
2     2022-02-11  Air India      AI       531    20:00    Delhi    24h 45m   
3     2022-02-11  Air India      AI       839    21:25    Delhi    26h 30m   
4     2022-02-11  Air India      AI       544    17:15    Delhi    06h 40m   
...          ...        ...     ...       ...      ...      ...        ...   
93482 2022-03-31    Vistara      UK       822    09:45  Chennai    10h 05m   
93483 2022-03-31    Vistara      UK       826    12:30  Chennai    10h 25m   
93484 2022-03-31    Vistara      UK       832    07:05  Chennai    13h 50m   
93485 2022-03-31    Vistara      UK       828    07:00  Chennai    10h 00m   
93486 2022-03-31    Vistara      UK       822    09:45  Chennai    10h 05m   

            stop arr_time         to   price  
0      non-stop 

In [15]:
print(EconomyData['time_taken'].unique())


['02h 10m' '02h 20m' '02h 15m' '02h 05m' '12h 15m' '16h 20m' '11h 45m'
 '14h 30m' '15h 40m' '03h 45m' '02h 30m' '05h 50m' '08h 00m' '06h 00m'
 '14h 40m' '16h 10m' '18h 00m' '23h 10m' '24h 10m' '08h 50m' '04h 30m'
 '15h 15m' '11h 00m' '19h 05m' '22h 50m' '26h 25m' '17h 45m' '19h 35m'
 '26h 40m' '15h 10m' '20h 50m' '11h 25m' '22h 15m' '26h 00m' '21h 45m'
 '03h 50m' '04h 25m' '07h 40m' '08h 20m' '10h 25m' '23h 45m' '19h 30m'
 '06h 30m' '12h 25m' '21h 05m' '28h 10m' '28h 15m' '09h 15m' '17h 55m'
 '07h 05m' '13h 50m' '07h 35m' '15h 50m' '24h 25m' '04h 10m' '04h 15m'
 '05h 05m' '29h 20m' '17h 00m' '27h 10m' '24h 45m' '05h 45m' '12h 45m'
 '13h 45m' '17h 50m' '05h 30m' '23h 50m' '05h 00m' '26h 30m' '12h 50m'
 '08h 55m' '11h 10m' '12h 10m' '15h 35m' '15h 45m' '07h 55m' '13h 15m'
 '16h 00m' '22h 45m' '06h 20m' '07h 15m' '30h 05m' '18h 15m' '06h 05m'
 '02h 00m' '12h 20m' '03h 30m' '10h 15m' '14h 10m' '25h 35m' '04h 05m'
 '09h 45m' '06h 40m' '09h 40m' '10h 05m' '12h 35m' '07h 00m' '08h 15m'
 '15h 

In [16]:
BusinessData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93487 entries, 0 to 93486
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        93487 non-null  datetime64[ns]
 1   airline     93487 non-null  object        
 2   ch_code     93487 non-null  object        
 3   num_code    93487 non-null  int64         
 4   dep_time    93487 non-null  object        
 5   from        93487 non-null  object        
 6   time_taken  93487 non-null  object        
 7   stop        93487 non-null  object        
 8   arr_time    93487 non-null  object        
 9   to          93487 non-null  object        
 10  price       93487 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(9)
memory usage: 7.8+ MB


In [17]:
EconomyData.head(10)
EconomyData= pd.DataFrame(EconomyData)
EconomyData['stop']= EconomyData['stop'].str.replace(r'\n.*','',regex=True)
EconomyData['date']=pd.to_datetime(EconomyData['date'], format="%d-%m-%Y", dayfirst=True)
print(EconomyData)

             date   airline ch_code  num_code dep_time     from time_taken  \
0      2022-02-11  SpiceJet      SG      8709    18:55    Delhi    02h 10m   
1      2022-02-11  SpiceJet      SG      8157    06:20    Delhi    02h 20m   
2      2022-02-11   AirAsia      I5       764    04:25    Delhi    02h 10m   
3      2022-02-11   Vistara      UK       995    10:20    Delhi    02h 15m   
4      2022-02-11   Vistara      UK       963    08:50    Delhi    02h 20m   
...           ...       ...     ...       ...      ...      ...        ...   
206769 2022-03-31   Vistara      UK       832    07:05  Chennai    13h 50m   
206770 2022-03-31   Vistara      UK       832    07:05  Chennai    13h 50m   
206771 2022-03-31   Vistara      UK       826    12:30  Chennai    20h 35m   
206772 2022-03-31   Vistara      UK       822    09:45  Chennai    23h 20m   
206773 2022-03-31   Vistara      UK       824    20:30  Chennai    24h 25m   

             stop arr_time         to  price  
0       non-stop

In [18]:
def convert_time_to_hours(time_str):
   
    if not isinstance(time_str,str) or time_str.strip()=='':
        return None
    hours=0
    minutes =0
    if 'h' in time_str:
        try:
            hours = float(time_str.split('h')[0].strip())
        except ValueError:
            hours = 0 
        
    if 'm' in time_str:
        try:
            minutes_part = time_str.split('h')[-1].replace('m', '').strip()
            if minutes_part:
                minutes = int(minutes_part)
        except ValueError:
            minutes = 0 
    return round(hours + (minutes / 60), 2)

# ***********
EconomyData['time_taken']= EconomyData['time_taken'].apply(convert_time_to_hours)
print(EconomyData)   
      

             date   airline ch_code  num_code dep_time     from  time_taken  \
0      2022-02-11  SpiceJet      SG      8709    18:55    Delhi        2.17   
1      2022-02-11  SpiceJet      SG      8157    06:20    Delhi        2.33   
2      2022-02-11   AirAsia      I5       764    04:25    Delhi        2.17   
3      2022-02-11   Vistara      UK       995    10:20    Delhi        2.25   
4      2022-02-11   Vistara      UK       963    08:50    Delhi        2.33   
...           ...       ...     ...       ...      ...      ...         ...   
206769 2022-03-31   Vistara      UK       832    07:05  Chennai       13.83   
206770 2022-03-31   Vistara      UK       832    07:05  Chennai       13.83   
206771 2022-03-31   Vistara      UK       826    12:30  Chennai       20.58   
206772 2022-03-31   Vistara      UK       822    09:45  Chennai       23.33   
206773 2022-03-31   Vistara      UK       824    20:30  Chennai       24.42   

             stop arr_time         to  price  
0   

In [19]:
BusinessData['time_taken']= BusinessData['time_taken'].apply(convert_time_to_hours)
print(BusinessData) 

            date    airline ch_code  num_code dep_time     from  time_taken  \
0     2022-02-11  Air India      AI       868    18:00    Delhi        2.00   
1     2022-02-11  Air India      AI       624    19:00    Delhi        2.25   
2     2022-02-11  Air India      AI       531    20:00    Delhi       24.75   
3     2022-02-11  Air India      AI       839    21:25    Delhi       26.50   
4     2022-02-11  Air India      AI       544    17:15    Delhi        6.67   
...          ...        ...     ...       ...      ...      ...         ...   
93482 2022-03-31    Vistara      UK       822    09:45  Chennai       10.08   
93483 2022-03-31    Vistara      UK       826    12:30  Chennai       10.42   
93484 2022-03-31    Vistara      UK       832    07:05  Chennai       13.83   
93485 2022-03-31    Vistara      UK       828    07:00  Chennai       10.00   
93486 2022-03-31    Vistara      UK       822    09:45  Chennai       10.08   

            stop arr_time         to   price  
0   

In [20]:
def convert_stop(stop_str):
    if pd.isna(stop_str) or not isinstance(stop_str, str) or stop_str.strip() == '':
        return None
    stop_str = str(stop_str).strip().lower() 
    if 'non' in stop_str:
        return 0
    try:
       return int(stop_str.split('-')[0].strip())
         
    except ValueError:
        return None

# **********

In [21]:
EconomyData['price']= EconomyData['price'].astype(str).replace(',','',regex=True).str.extract('(\d+\.?\d*)')[0].astype(float)
EconomyData['stop']= EconomyData['stop'].apply(convert_stop).astype('Int64')
print(EconomyData)

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_380056/916015671.py:1: SyntaxWarning: invalid escape sequence '\d'
  EconomyData['price']= EconomyData['price'].astype(str).replace(',','',regex=True).str.extract('(\d+\.?\d*)')[0].astype(float)


             date   airline ch_code  num_code dep_time     from  time_taken  \
0      2022-02-11  SpiceJet      SG      8709    18:55    Delhi        2.17   
1      2022-02-11  SpiceJet      SG      8157    06:20    Delhi        2.33   
2      2022-02-11   AirAsia      I5       764    04:25    Delhi        2.17   
3      2022-02-11   Vistara      UK       995    10:20    Delhi        2.25   
4      2022-02-11   Vistara      UK       963    08:50    Delhi        2.33   
...           ...       ...     ...       ...      ...      ...         ...   
206769 2022-03-31   Vistara      UK       832    07:05  Chennai       13.83   
206770 2022-03-31   Vistara      UK       832    07:05  Chennai       13.83   
206771 2022-03-31   Vistara      UK       826    12:30  Chennai       20.58   
206772 2022-03-31   Vistara      UK       822    09:45  Chennai       23.33   
206773 2022-03-31   Vistara      UK       824    20:30  Chennai       24.42   

        stop arr_time         to   price  
0       

In [22]:
BusinessData['price']= BusinessData['price'].astype(str).replace(',','',regex=True).str.extract('(\d+\.?\d*)')[0].astype(float)
BusinessData['stop']= BusinessData['stop'].apply(convert_stop).astype('Int64') ## Int64 support None value

print(BusinessData)     

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_380056/396972601.py:1: SyntaxWarning: invalid escape sequence '\d'
  BusinessData['price']= BusinessData['price'].astype(str).replace(',','',regex=True).str.extract('(\d+\.?\d*)')[0].astype(float)


            date    airline ch_code  num_code dep_time     from  time_taken  \
0     2022-02-11  Air India      AI       868    18:00    Delhi        2.00   
1     2022-02-11  Air India      AI       624    19:00    Delhi        2.25   
2     2022-02-11  Air India      AI       531    20:00    Delhi       24.75   
3     2022-02-11  Air India      AI       839    21:25    Delhi       26.50   
4     2022-02-11  Air India      AI       544    17:15    Delhi        6.67   
...          ...        ...     ...       ...      ...      ...         ...   
93482 2022-03-31    Vistara      UK       822    09:45  Chennai       10.08   
93483 2022-03-31    Vistara      UK       826    12:30  Chennai       10.42   
93484 2022-03-31    Vistara      UK       832    07:05  Chennai       13.83   
93485 2022-03-31    Vistara      UK       828    07:00  Chennai       10.00   
93486 2022-03-31    Vistara      UK       822    09:45  Chennai       10.08   

       stop arr_time         to    price  
0       

In [23]:
BusinessData['flight_class'] = 'Business'
EconomyData['flight_class'] = 'Economy'

In [24]:
CleanedData = pd.concat([BusinessData,EconomyData],ignore_index=True)
  

In [25]:
CleanedData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300261 entries, 0 to 300260
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   date          300261 non-null  datetime64[ns]
 1   airline       300261 non-null  object        
 2   ch_code       300261 non-null  object        
 3   num_code      300261 non-null  int64         
 4   dep_time      300261 non-null  object        
 5   from          300261 non-null  object        
 6   time_taken    300261 non-null  float64       
 7   stop          286973 non-null  Int64         
 8   arr_time      300261 non-null  object        
 9   to            300261 non-null  object        
 10  price         300261 non-null  float64       
 11  flight_class  300261 non-null  object        
dtypes: Int64(1), datetime64[ns](1), float64(2), int64(1), object(7)
memory usage: 27.8+ MB


In [26]:
CleanedDatam =CleanedData.rename(columns={'from':'source_city','to':'destination_city'}, inplace=True)
CleanedData.columns

Index(['date', 'airline', 'ch_code', 'num_code', 'dep_time', 'source_city',
       'time_taken', 'stop', 'arr_time', 'destination_city', 'price',
       'flight_class'],
      dtype='object')

In [29]:
pipeline = dlt.pipeline(
    pipeline_name = 'flight_price',
    destination = "duckdb",
    dataset_name = "flight_prediction",
    )
info_CD = pipeline.run(CleanedData,table_name="CleanedData", write_disposition="replace")
info_BD = pipeline.run(BusinessData,table_name="BusinessData", write_disposition="replace")
info_ED = pipeline.run(EconomyData,table_name="EconomyData", write_disposition="replace")
# info = pipeline.run([CleanedData,BusinessData,EconomyData],write_disposition="replace")
print(info_CD,info_BD,info_ED)

Pipeline flight_price load step completed in 0.44 seconds
1 load package(s) were loaded to destination duckdb and into dataset flight_prediction
The duckdb destination used duckdb:////home/nasrin/Zoomcamp-2025./project/flight_price.duckdb location to store data
Load package 1743513244.5574107 is LOADED and contains no failed jobs Pipeline flight_price load step completed in 0.40 seconds
1 load package(s) were loaded to destination duckdb and into dataset flight_prediction
The duckdb destination used duckdb:////home/nasrin/Zoomcamp-2025./project/flight_price.duckdb location to store data
Load package 1743513245.302497 is LOADED and contains no failed jobs Pipeline flight_price load step completed in 0.42 seconds
1 load package(s) were loaded to destination duckdb and into dataset flight_prediction
The duckdb destination used duckdb:////home/nasrin/Zoomcamp-2025./project/flight_price.duckdb location to store data
Load package 1743513245.8319108 is LOADED and contains no failed jobs


In [30]:
print(pipeline.last_trace)

Run started at 2025-04-01 13:14:05.817117+00:00 and COMPLETED in 0.60 seconds with 4 steps.
Step extract COMPLETED in 0.14 seconds.

Load package 1743513245.8319108 is EXTRACTED and NOT YET LOADED to the destination and contains no failed jobs

Step normalize COMPLETED in 0.02 seconds.
Normalized data for the following tables:
- economy_data: 206774 row(s)

Load package 1743513245.8319108 is NORMALIZED and NOT YET LOADED to the destination and contains no failed jobs

Step load COMPLETED in 0.43 seconds.
Pipeline flight_price load step completed in 0.42 seconds
1 load package(s) were loaded to destination duckdb and into dataset flight_prediction
The duckdb destination used duckdb:////home/nasrin/Zoomcamp-2025./project/flight_price.duckdb location to store data
Load package 1743513245.8319108 is LOADED and contains no failed jobs

Step run COMPLETED in 0.60 seconds.
Pipeline flight_price load step completed in 0.42 seconds
1 load package(s) were loaded to destination duckdb and into da

In [31]:
pipeline.dataset(dataset_type="default").CleanedData.df()

,date,airline,ch_code,num_code,dep_time,source_city,time_taken,stop,arr_time,destination_city,price,flight_class
0,2022-02-11,Air India,AI,868,18:00,Delhi,2.00,0,20:00,Mumbai,25612.0,Business
1,2022-02-11,Air India,AI,624,19:00,Delhi,2.25,0,21:15,Mumbai,25612.0,Business
2,2022-02-11,Air India,AI,531,20:00,Delhi,24.75,1,20:45,Mumbai,42220.0,Business
3,2022-02-11,Air India,AI,839,21:25,Delhi,26.50,1,23:55,Mumbai,44450.0,Business
4,2022-02-11,Air India,AI,544,17:15,Delhi,6.67,1,23:55,Mumbai,46690.0,Business
...,...,...,...,...,...,...,...,...,...,...,...,...
300256,2022-03-31,Vistara,UK,832,07:05,Chennai,13.83,1,20:55,Hyderabad,7697.0,Economy
300257,2022-03-31,Vistara,UK,832,07:05,Chennai,13.83,1,20:55,Hyderabad,7709.0,Economy
300258,2022-03-31,Vistara,UK,826,12:30,Chennai,20.58,1,09:05,Hyderabad,8640.0,Economy
300259,2022-03-31,Vistara,UK,822,09:45,Chennai,23.33,1,09:05,Hyderabad,8640.0,Economy


In [32]:
pipeline.dataset(dataset_type="default").BusinessData.df()

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price,flight_class
0,2022-02-11 00:00:00,Air India,AI,868,18:00,Delhi,2.0,0,20:00,Mumbai,25612.0,Business
1,2022-02-11 00:00:00,Air India,AI,624,19:00,Delhi,2.25,0,21:15,Mumbai,25612.0,Business
2,2022-02-11 00:00:00,Air India,AI,531,20:00,Delhi,24.75,1,20:45,Mumbai,42220.0,Business
3,2022-02-11 00:00:00,Air India,AI,839,21:25,Delhi,26.5,1,23:55,Mumbai,44450.0,Business
4,2022-02-11 00:00:00,Air India,AI,544,17:15,Delhi,6.67,1,23:55,Mumbai,46690.0,Business
...,...,...,...,...,...,...,...,...,...,...,...,...
93482,2022-03-31 00:00:00,Vistara,UK,822,09:45,Chennai,10.08,1,19:50,Hyderabad,69265.0,Business
93483,2022-03-31 00:00:00,Vistara,UK,826,12:30,Chennai,10.42,1,22:55,Hyderabad,77105.0,Business
93484,2022-03-31 00:00:00,Vistara,UK,832,07:05,Chennai,13.83,1,20:55,Hyderabad,79099.0,Business
93485,2022-03-31 00:00:00,Vistara,UK,828,07:00,Chennai,10.0,1,17:00,Hyderabad,81585.0,Business


In [37]:
pipeline.dataset(dataset_type="default").EconomyData.df()

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price
0,2022-02-11 00:00:00,SpiceJet,SG,8709,18:55,Delhi,02h 10m,non-stop,21:05,Mumbai,"5,953"
1,2022-02-11 00:00:00,SpiceJet,SG,8157,06:20,Delhi,02h 20m,non-stop,08:40,Mumbai,"5,953"
2,2022-02-11 00:00:00,AirAsia,I5,764,04:25,Delhi,02h 10m,non-stop,06:35,Mumbai,"5,956"
3,2022-02-11 00:00:00,Vistara,UK,995,10:20,Delhi,02h 15m,non-stop,12:35,Mumbai,"5,955"
4,2022-02-11 00:00:00,Vistara,UK,963,08:50,Delhi,02h 20m,non-stop,11:10,Mumbai,"5,955"
...,...,...,...,...,...,...,...,...,...,...,...
206769,2022-03-31 00:00:00,Vistara,UK,832,07:05,Chennai,13h 50m,1-stop,20:55,Hyderabad,"7,697"
206770,2022-03-31 00:00:00,Vistara,UK,832,07:05,Chennai,13h 50m,1-stop,20:55,Hyderabad,"7,709"
206771,2022-03-31 00:00:00,Vistara,UK,826,12:30,Chennai,20h 35m,1-stop,09:05,Hyderabad,"8,640"
206772,2022-03-31 00:00:00,Vistara,UK,822,09:45,Chennai,23h 20m,1-stop,09:05,Hyderabad,"8,640"


In [33]:
with pipeline.sql_client() as client:
    res = client.execute_sql(
            """
            SELECT stop FROM business_data WHERE stop==('\n');
            
            """
        )
    print(res)

[]


In [34]:
pipeline = dlt.pipeline(
    pipeline_name = 'flight_price',
    destination = "bigquery",
    dataset_name = "flight_prediction", # <--- to run pipeline in production
    )

2025-04-01 15:14:42,509|[WARNING]|380056|140428028280960|dlt|pipeline.py|_state_to_props:1640|The destination dlt.destinations.duckdb:None in state differs from destination dlt.destinations.bigquery:bigquery in pipeline and will be ignored


In [35]:
!pip install dlt google-cloud-bigquery


In [36]:
!pip install google-cloud-bigquery-storage

In [37]:
!export GOOGLE_APPLICATION_CREDENTIALS="./bq_key.json"

In [39]:
pipeline = dlt.pipeline(
    pipeline_name="flight_price",
    destination="bigquery",
    dataset_name="flight_prediction",
    dev_mode=True,
)

infoCD = pipeline.run(CleanedData,table_name="CleanedData", write_disposition="replace")
infoBD = pipeline.run(BusinessData,table_name="BusinessData", write_disposition="replace")
infoED = pipeline.run(EconomyData,table_name="EconomyData", write_disposition="replace")

print(infoCD,infoBD,infoED)

2025-04-01 15:17:52,188|[WARNING]|380056|140428028280960|dlt|type_mapping.py|to_db_datetime_type:56|Column flags for timezone or precision are not yet supported in this destination. One or both of these flags were used in column 'date'. in table 'cleaned_data'.
2025-04-01 15:17:54,715|[WARNING]|380056|140428028280960|dlt|type_mapping.py|to_db_datetime_type:56|Column flags for timezone or precision are not yet supported in this destination. One or both of these flags were used in column 'date'. in table 'cleaned_data'.
2025-04-01 15:18:10,488|[WARNING]|380056|140428028280960|dlt|type_mapping.py|to_db_datetime_type:56|Column flags for timezone or precision are not yet supported in this destination. One or both of these flags were used in column 'date'. in table 'cleaned_data'.
2025-04-01 15:18:10,489|[WARNING]|380056|140428028280960|dlt|type_mapping.py|to_db_datetime_type:56|Column flags for timezone or precision are not yet supported in this destination. One or both of these flags were 

Pipeline flight_price load step completed in 18.08 seconds
1 load package(s) were loaded to destination bigquery and into dataset flight_prediction_20250401011751
The bigquery destination used None@kestra-sandbox-449808 location to store data
Load package 1743513471.81282 is LOADED and contains no failed jobs Pipeline flight_price load step completed in 13.03 seconds
1 load package(s) were loaded to destination bigquery and into dataset flight_prediction_20250401011751
The bigquery destination used None@kestra-sandbox-449808 location to store data
Load package 1743513490.2915576 is LOADED and contains no failed jobs Pipeline flight_price load step completed in 13.80 seconds
1 load package(s) were loaded to destination bigquery and into dataset flight_prediction_20250401011751
The bigquery destination used None@kestra-sandbox-449808 location to store data
Load package 1743513503.540441 is LOADED and contains no failed jobs
